In [1]:
import os
import typing

# Dask Configuration

In [2]:
from dask_jobqueue import PBSCluster
from pathlib import Path

# Define the working directory path
working_directory = str(Path.cwd())

# Launch a scheduler and workers on HPC via PBS
cluster = PBSCluster(
     cores=4,
     memory="8GB",
     processes=1,     # TODO - was set to 1 prior to talking to Yehuda 
     queue="tamirQ",
     walltime="05:30:00",
     scheduler_options={"dashboard_address": ":12435"},
     # Additional custom options
     log_directory="dask-logs",
     #worker_extra_args=["--lifetime", "25m", "--lifetime-stagger", "4m"],  # for walltime="00:30:00"
     job_script_prologue=[f"cd {working_directory}"]
)

In [3]:
cluster

Dashboard: http://132.66.112.146:12435/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://132.66.112.146:33311,Workers: 0
Dashboard: http://132.66.112.146:12435/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
cluster.adapt(minimum=30, maximum=60)
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -q tamirQ
#PBS -l select=1:ncpus=4:mem=7630MB
#PBS -l walltime=05:30:00
#PBS -e dask-logs/
#PBS -o dask-logs/
cd /tamir2/moranb/microbiome/Igem_TAU_2021
/tamir2/moranb/microbiome/Igem_TAU_2021/venv/bin/python -m distributed.cli.dask_worker tcp://132.66.112.146:33311 --nthreads 4 --memory-limit 7.45GiB --name dummy-name --nanny --death-timeout 60



In [5]:
from dask.distributed import Client, progress, wait, get_client, get_worker
client = Client(cluster)

In [6]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://132.66.112.146:12435/status,
Dashboard: http://132.66.112.146:12435/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://132.66.112.146:33311,Workers: 0
Dashboard: http://132.66.112.146:12435/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
import dask.bag as db
import dask.dataframe as dd
from dask import delayed, compute, persist
import json
from collections import defaultdict
import matplotlib
import numpy as np
import pandas as pd
import re


# Analysis for homo sapiens genome

In [8]:
from Bio import SeqIO
from analysis.input_testing_data.generate_input_testing_data_for_modules import generate_testing_data
from analysis.input_testing_data.generate_input_testing_data_for_modules import generate_testing_data_for_ecoli_and_bacillus

from modules.main import run_modules

In [9]:
output_path = "/tamir2/moranb/microbiome/Igem_TAU_2021/analysis/results/homo_sapiens"

In [10]:
def get_orf_summary(summary: typing.Dict[str, typing.Any], evaluation_method: str = "average_distance_score") -> typing.Dict[str, typing.Any]:
    if len(summary["evaluation"]) == 1:
        return summary["orf"]
    final_evaluation = summary["final_evaluation"]
    for i, evaluation_summary in enumerate(summary["evaluation"]):
        if evaluation_summary[evaluation_method] == final_evaluation[evaluation_method]:
            return summary["orf"][i]

In [11]:
def get_total_run_time(summary: typing.Dict[str, typing.Any]) -> float:
     if len(summary["evaluation"]) == 1:
        return summary["orf"]["run_time"]
     run_time = 0
     for orf_summary in summary["orf"]:
         run_time += orf_summary["run_time"]
     return run_time

In [12]:
def convert_to_json_result(x):
    gene_name = x[0]
    result = run_modules(x[1], should_run_output_module=False)
    return {"gene_name": gene_name, "summary": result}
    
    # orf = get_orf_summary(result)
    # if "orf" not in result or orf is None: # probably thrown in case of an error
    #     return {"error": result}
    # run_time = get_total_run_time(result)
    # iterations_count = orf.get("iterations_count", 1)
    # return {
    #     "initial_optimization_score": orf.get("initial_sequence_optimization_score"),
    #     "final_optimization_score": orf.get("final_sequence_optimization_score"),
    #     "average_distance_score": result["final_evaluation"].get("average_distance_score"),
    #     "average_distance_non_normalized_score": result["final_evaluation"].get("average_distance_non_normalized_score"),
    #     "weakest_link_score": result["final_evaluation"].get("weakest_link_score"),
    #     "ratio_score": result["final_evaluation"].get("ratio_score"),
    #     "gene_name": gene_name,
    #     "orf_optimization_cub_index": result["module_input"].get("orf_optimization_cub_index"),
    #     "evaluation_score": result["module_input"].get("evaluation_score"),
    #     "run_time": run_time,
    #     "iterations_count": iterations_count,
    # }

In [13]:
def parse_nested_brackets(s):
    stack = []
    current = ""
    result = []

    for char in s:
        if char == '[':
            if stack:
                current += char
            stack.append('[')
        elif char == ']':
            stack.pop()
            if stack:
                current += char
            else:
                result.append(current.strip())
                current = ""
        else:
            if stack:
                current += char

    return result

In [14]:
def extract_ncbi_sequences_for_analysis(fasta_file_path: str) -> None:
    with open(fasta_file_path, "r") as fasta_handle:
        genome_dict = SeqIO.to_dict(SeqIO.parse(fasta_handle, "fasta"))

    print(f"Total number of sequences in file: {len(genome_dict)}")
    genes_with_wrong_length = []
    genes_with_invalid_chars = []
    partial_genes_or_segments = []
    very_short_genes = []
    short_genes = []

    records = []
    for record, value in genome_dict.items():
        # NC is the RefSeq prefix for Complete genome or chromosome
        # if "NC" not in record:
        #     continue

        if len(value.seq) % 3 != 0:
            genes_with_wrong_length.append(record)
            continue

        initiation_prefix_length = 15
        min_sequence_length = 10
        if len(value.seq) <= initiation_prefix_length * 3:
            very_short_genes.append(record)
            continue
        if len(value.seq) <= (initiation_prefix_length + min_sequence_length) * 3:
            short_genes.append(record)
            continue

        # Can be found due to partial segments or contigs
        if not value.seq.startswith("ATG"):
            partial_genes_or_segments.append(record)
            continue

        if any(x not in ["A", "C", "T", "G"] for x in str(value.seq)):
            print(F"Skip sequence {value.seq} for {record} because it contains an invalid character for CDS")
            genes_with_invalid_chars.append(record)
            continue

        record_description = value.description
        parameters = parse_nested_brackets(value.description)
        parameters_dict = {}
        for param in parameters:
            parsed_parameter = param.split("=")
            parameters_dict[parsed_parameter[0]] = parsed_parameter[1]
            parameters_dict["sequence"] = str(value.seq)

        records.append(parameters_dict)

    print(f"Skipped {len(genes_with_wrong_length)} genes with length not divisible by 3")
    print(f"Skipped {len(very_short_genes)} genes with length shorter than the initiation prefix")
    print(f"Skipped {len(short_genes)} genes with length that is too short")
    print(f"Skipped {len(partial_genes_or_segments)} genes that don't start with a start codon")
    
    genes_df = pd.DataFrame(records)
    return genes_df

In [15]:
homo_sapiens_fasta = "/tamir2/moranb/microbiome/homo_sapiens_genome/cds_from_genomic.fna"
# gene_records = extract_ncbi_sequences_for_analysis(homo_sapiens_fasta)
# gene_records

In [16]:
# filtered_gene_records = gene_records.loc[gene_records.groupby("gene")["sequence"].apply(lambda x: x.str.len().idxmax())]
# filtered_gene_records

In [17]:
# filtered_gene_records.to_pickle("homo_sapiens_gene_records.pkl")
filtered_gene_records = pd.read_pickle("homo_sapiens_gene_records.pkl")

## Bacillus and E.coli

In [18]:
optimization_cub_index = "CAI"
is_ecoli_optimized = False # True
batch_size = 250

    
for optimization_method in [
    # "single_codon_diff", 
    # "single_codon_ratio", 
    # "zscore_bulk_aa_diff",
    # "zscore_bulk_aa_ratio",
    # "zscore_single_aa_diff",
    "zscore_single_aa_ratio",
    "single_wanted_organism",
]:
    configuration = f"e_coli_optimized_{is_ecoli_optimized}_bacillus_optimized_{not is_ecoli_optimized}"
    configuration_output_path = os.path.join(output_path, configuration)
    
    inputs = [(row.gene, 
               generate_testing_data_for_ecoli_and_bacillus(
                   orf_optimization_method=optimization_method,
                   orf_optimization_cub_index=optimization_cub_index,
                   is_ecoli_optimized=is_ecoli_optimized,
                   tuning_param=0.5, 
                   sequence=row.sequence,
                   output_path=os.path.join(configuration_output_path, row.gene),
                   evaluation_score="average_distance",
                   initiation_optimization_method="original",
               )) for row in filtered_gene_records.itertuples(index=False)]
    
    for batch_index, batch_start_index in enumerate(range(0, len(inputs), batch_size)):
        if batch_index < 80 and optimization_method == "zscore_single_aa_ratio":
             continue
        inputs_batch = inputs[batch_start_index: batch_start_index+batch_size]
        inputs_db = db.from_sequence(inputs_batch)
        results_db = inputs_db.map(convert_to_json_result)
        batch_file_path = os.path.join(configuration_output_path,optimization_method, f"batch-{batch_index}")
        batch_file_path = batch_file_path + "-debug"
        results_db.map(json.dumps).to_textfiles(os.path.join(batch_file_path, '*.json'))
        os.mknod(os.path.join(batch_file_path, "done"))

Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, ini